vision: I wanted to make a code that turns a black and white photo into colored photo but changed into 
DayDream project. In Daydream project, I used the fchollet's demo as a base. In the project, we will 
be able to see the photos into "dayDreamlike fashion.

Implementation: In this project, we use
deined list of scales to process the images. Each successive scale will have larger image than the previous
one( about 40% larger). To maximize the loss defined, the program will run gradient ascent.
after the ascent, the program will reinject lost details into the image to avoid blur or pixelated images.


In [1]:
from keras.applications import inception_v3
from keras import backend as K
import numpy as np
import scipy
import matplotlib.pyplot as plt
from keras.preprocessing import image
from utils import *
%matplotlib inline

Using TensorFlow backend.


In [2]:
K.set_learning_phase(0) 
model = inception_v3.InceptionV3(weights='imagenet', include_top=False)

In [3]:
model.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, None, 3 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, None, None, 3 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (


I made some changes into the filters that produced some interesting patterns.
I modified the number of layers, the weights of the layers, and the actual layers used.

In [7]:
layer_contributions = {
    'conv2d_52': 10.,
    'mixed5': 5.,
    'mixed6': 2.,
    }

In [8]:
##########################################
# RUN THIS CELL TO UPDATE THE GRAPH
# NO NEED TO CHANGE THIS CODE


layer_dict = dict([(layer.name, layer) for layer in model.layers]) # map layer_name -> instance

loss = K.variable(0.) # initialize loss to 0
for layer_name in layer_contributions:
    coeff = layer_contributions[layer_name]
    activation = layer_dict[layer_name].output
    scaling = K.prod(K.cast(K.shape(activation), 'float32')) # number of activation in a layer
    # avoid border effect by selecting 2:-2 
    loss += coeff * K.sum(K.square(activation[:,2:-2, 2:-2, :])) / scaling
    # Gradient ascent
    
dream = model.input
grads = K.gradients(loss, dream)[0]  # gradient of the input with regard to the loss.
grads /= K.maximum(K.mean(K.abs(grads)), 1e-7) # normalize gradient (like clipping)

outputs = [loss, grads]
fetch_loss_and_grads = K.function([dream], outputs)

def eval_loss_and_grads(x):
    outs = fetch_loss_and_grads([x])
    loss_value = outs[0]
    grad_values = outs[1]
    return loss_value, grad_values
   
def gradient_ascent(picture, iterations, learning_rate, max_loss=None):
    for i in range(iterations):
        loss_value, gradient_values = eval_loss_and_grads(picture)
        if (max_loss!=None) and (loss_value>max_loss):
            # Stop the algorithm after a Maximum loss threshold 
            break
        print('...Loss value at', i, ':', loss_value)
        picture = picture + learning_rate*gradient_values
    return picture

In [9]:
# running gradient ascent of different sceles of image
def main(base_image_path):
    learning_rate = 0.01
    num_octave = 3
    octave_scale = 1.4
    iterations = 20
    max_loss = 10 

    img = preprocess_image(base_image_path)
    original_shape = img.shape[1:3]
    successive_shapes = [original_shape]
    for i in range(1, num_octave):
        shape = tuple([int(dim / (octave_scale ** i)) for dim in original_shape])
        successive_shapes.append(shape)
    successive_shapes = successive_shapes[::-1]
    original_img = np.copy(img)
    shrunk_original_img = resize_img(img, successive_shapes[0]) # srunk to first scale

    for shape in successive_shapes:
        print('Processing image shape', shape)
        img = resize_img(img, shape)
        img = gradient_ascent(img,
                              iterations=iterations,
                              learning_rate=learning_rate,
                              max_loss=max_loss)
        upscaled_shrunk_original_img = resize_img(shrunk_original_img, shape)
        same_size_original = resize_img(original_img, shape)
        lost_detail = same_size_original - upscaled_shrunk_original_img
        img += lost_detail
        shrunk_original_img = resize_img(original_img, shape)
        save_img(img, fname='output/dream_at_scale_' + str(shape) + '.png')
    save_img(img, fname='output/final_dream.png')

In [10]:
main('./sky.JPG')

Processing image shape (366, 653)
...Loss value at 0 : 2.448592
...Loss value at 1 : 3.6646802
...Loss value at 2 : 5.17015
...Loss value at 3 : 6.728642
...Loss value at 4 : 8.385723
...Loss value at 5 : 9.875048
Processing image shape (512, 914)
...Loss value at 0 : 4.475424
...Loss value at 1 : 7.306331
...Loss value at 2 : 8.684753
Processing image shape (718, 1280)
...Loss value at 0 : 4.304409
...Loss value at 1 : 6.9515243
...Loss value at 2 : 8.382611


I made some changes that made the code faster. I modified the max loss to produce dramatic changes. 
I modified learning rate for speed up, and I modified iterations to increase loss.

In [15]:
layer_contributions = {
    'conv2d_52': 25.,
    }

In [16]:
####################################
# RUN THIS CELL TO UPDATE THE GRAPH
# NO NEED TO CHANGE THIS CODE


layer_dict = dict([(layer.name, layer) for layer in model.layers]) # map layer_name -> instance

loss = K.variable(0.) # initialize loss to 0
for layer_name in layer_contributions:
    coeff = layer_contributions[layer_name]
    activation = layer_dict[layer_name].output
    scaling = K.prod(K.cast(K.shape(activation), 'float32')) # number of activation in a layer
    # avoid border effect by selecting 2:-2 
    loss += coeff * K.sum(K.square(activation[:,2:-2, 2:-2, :])) / scaling
    # Gradient ascent
    
dream = model.input
grads = K.gradients(loss, dream)[0]  # gradient of the input with regard to the loss.
grads /= K.maximum(K.mean(K.abs(grads)), 1e-7) # normalize gradient (like clipping)

outputs = [loss, grads]
fetch_loss_and_grads = K.function([dream], outputs)

def eval_loss_and_grads(x):
    outs = fetch_loss_and_grads([x])
    loss_value = outs[0]
    grad_values = outs[1]
    return loss_value, grad_values
   
def gradient_ascent(picture, iterations, learning_rate, max_loss=None):
    for i in range(iterations):
        loss_value, gradient_values = eval_loss_and_grads(picture)
        if (max_loss!=None) and (loss_value>max_loss):
            # Stop the algorithm after a Maximum loss threshold 
            break
        print('...Loss value at', i, ':', loss_value)
        picture = picture + learning_rate*gradient_values
    return picture

In [17]:

def main(base_image_path):
    learning_rate = 0.01
    num_octave = 3
    octave_scale = 1.4
    iterations = 20
    max_loss = 100

    img = preprocess_image(base_image_path)
    original_shape = img.shape[1:3]
    successive_shapes = [original_shape]
    for i in range(1, num_octave):
        shape = tuple([int(dim / (octave_scale ** i)) for dim in original_shape])
        successive_shapes.append(shape)
    successive_shapes = successive_shapes[::-1]
    original_img = np.copy(img)
    shrunk_original_img = resize_img(img, successive_shapes[0]) # srunk to first scale

    for shape in successive_shapes:
        print('Processing image shape', shape)
        img = resize_img(img, shape)
        img = gradient_ascent(img,
                              iterations=iterations,
                              learning_rate=learning_rate,
                              max_loss=max_loss)
        upscaled_shrunk_original_img = resize_img(shrunk_original_img, shape)
        same_size_original = resize_img(original_img, shape)
        lost_detail = same_size_original - upscaled_shrunk_original_img
        img += lost_detail
        shrunk_original_img = resize_img(original_img, shape)
        save_img(img, fname='output/dream_at_scale_' + str(shape) + '.png')
    save_img(img, fname='output/final_dream.png')

In [20]:
main('./sky.JPG')



Processing image shape (366, 653)
...Loss value at 0 : 4.4474545
...Loss value at 1 : 6.882981
...Loss value at 2 : 9.74409
...Loss value at 3 : 12.532616
...Loss value at 4 : 15.60267
...Loss value at 5 : 18.938816
...Loss value at 6 : 21.85651
...Loss value at 7 : 24.83817
...Loss value at 8 : 28.14423
...Loss value at 9 : 32.22948
...Loss value at 10 : 34.971424
...Loss value at 11 : 37.868378
...Loss value at 12 : 41.166203
...Loss value at 13 : 44.495213
...Loss value at 14 : 48.371857
...Loss value at 15 : 51.09748
...Loss value at 16 : 54.586388
...Loss value at 17 : 56.792393
...Loss value at 18 : 61.2232
...Loss value at 19 : 63.400944
Processing image shape (512, 914)
...Loss value at 0 : 11.042963
...Loss value at 1 : 20.593567
...Loss value at 2 : 25.513317
...Loss value at 3 : 30.28683
...Loss value at 4 : 33.489307
...Loss value at 5 : 38.288555
...Loss value at 6 : 41.162975
...Loss value at 7 : 45.552723
...Loss value at 8 : 49.066944
...Loss value at 9 : 52.121178
...L

results: The program used algorithm of upscaling the photos starting with a smaller image to create
dayDream like images. As I was doing research on the project, I found out that there was google version
of DayDream and the process they adopted was quite interesting. Google trained  their neural network
on a label dataset of images from cats to ships and what else. As they trained, they built internal 
representations at each layer. Eventually, the first few layers learned low level images like lines and 
edges, the higher levels learned eys, faces, and etc, progressively getting more abstract. The way they
daydream an image is not to classify a new image, but to maximize similarity between the image and a 
given layer. Fchollet's algorithm was much more intuitive and simpler to adopt but Google's implementation
was able to learn much more "dayDream like" images.



Implications: Although the project was for images, it could be adopted to create videos also. 
If the google version of the dayDream could be made into videos, it might be possible to help drug
addicts. We could have people with addiction problems on drugs that give hallucination and dayDreams,
to provide the guide line for the images of what they saw and train the neural network from those 
images to create videos to help addicts to use the videos instead of the drugs.